In [34]:
import pickle
import pandas as pd

from util.load_scenarios import load_scenario_df
from util.load_evaluations import load_rs_df

## Load data


In [35]:
rs_df = load_rs_df("../data/evaluation/random_search")
rs_df = rs_df.reset_index().set_index(
    ['fps', 'highquality', 'rep', 'it', 'route_id'])

scenario_df = load_scenario_df("../data/definition/rs_routes.xml")

df = rs_df.join(scenario_df, on='route_index', how='inner').sort_index()
# df

In [36]:

with open("model.pkl", "rb") as f:
    clf = pickle.load(f)

## Compute solutions


In [45]:
# Do it just for 10FPS epic
df = df.xs(("10", "True"), drop_level=False).sort_index()

In [46]:

def get_solutions_of_iteration(df, curr_it):

    idx = pd.IndexSlice
    # get slice of data frame from iteration 0 to current iteration
    slice = df.loc[idx[:, :, :, :curr_it+1], :]

    # get index of minumum driving score -> solution (in iterations so far)
    gb = slice.groupby(['fps', 'highquality', 'rep',])[
        'driving_score']

    solutions_idx = gb.idxmin().rename('solution_idx')

    # get value of minimum driving score
    solutions_score = gb.min().rename('solution_score')

    solutions = pd.concat([solutions_idx, solutions_score], axis=1)
    solutions['it'] = curr_it
    solutions = solutions.reset_index().set_index(
        ['fps', 'highquality', 'rep', 'it'])

    return solutions

In [47]:
def get_rs_solutions(df) -> pd.DataFrame:

    NUM_OF_ITERAITONS = 25

    sols_df = pd.concat([get_solutions_of_iteration(df, it)
                        for it in range(NUM_OF_ITERAITONS)], axis=0)

    # at what iteration did we find the solution
    sols_df['solution_it'] = sols_df['solution_idx'].apply(lambda x: x[-2])
    # what is the route_id of the solution
    sols_df['solution_route_id'] = sols_df['solution_idx'].apply(
        lambda x: x[-1])

    # we don't need that once unpacked
    sols_df = sols_df.drop('solution_idx', axis=1)

    # add data from verification
    # sols_df = sols_df.join(oracle_df, on='solution_route_id')
    sols_df = sols_df.sort_index()
    return sols_df


get_rs_solutions(df)

solution_score  solution_it  solution_route_id
fps highquality rep it                                                
10  True        0   0         0.191275            0   RouteScenario_88
                    1         0.191275            0   RouteScenario_88
                    2         0.110279            3  RouteScenario_317
                    3         0.110279            3  RouteScenario_317
                    4         0.110279            3  RouteScenario_317
...                                ...          ...                ...
                29  20        0.025790            6  RouteScenario_107
                    21        0.025790            6  RouteScenario_107
                    22        0.025790            6  RouteScenario_107
                    23        0.025790            6  RouteScenario_107
                    24        0.025790            6  RouteScenario_107

[750 rows x 3 columns]